In [1]:
import pandas as pd
data = pd.read_excel("datasets/meetreq_training.xlsx")

import win32com.client as win32
outlook = win32.Dispatch('outlook.application')

import os
from os.path import join

In [2]:
for train in data.training.unique():
    train_base = data[(data.training == train) & (data.status_meetreq.isnull())]
    for date_start in train_base.waktu.unique():
        date_base = train_base[train_base.waktu == date_start]
        mail = outlook.CreateItem(1)
        ts = pd.to_datetime(str(date_start)) 
        d = ts.strftime('%Y-%m-%d %H:%M')
        mail.Start = d
        mail.Subject = train
        mail.Duration = date_base.duration.unique()[0].item()
        mail.Location = date_base.email_meeting_room.unique()[0]
        mail.MeetingStatus = 1
        for trainee in date_base.email_peserta.unique():
            mail.Recipients.Add(trainee)
        for trainer in date_base.email_trainer.unique():
            mail.Recipients.Add(trainer)
        for room in date_base.email_meeting_room.unique():
            booking = mail.Recipients.Add(room)
            booking.resolve
        for carboncopy in date_base.email_optional.unique():
            cc = mail.Recipients.Add(carboncopy)
            cc.Type = 2
        body1 = ("Dear rekan-rekan,\n" \
"Mengundang rekan-rekan POK mengikuti:\n" \
"POK | %(judul)s \n" \
"\n" \
"Hari, Tanggal: %(hari)s, %(tanggal)s \n" \
"Durasi: %(durasi)i jam \n" \
"Tempat: %(tempat)s \n" \
"Trainer: %(trainer)s \n" \
"Trainee:\n" % {"judul": train,
                "tanggal": d,
                "durasi": date_base.duration.unique()[0].item()/60,
                "hari": date_base.hari.unique()[0],
                "tempat": date_base.meeting_room.unique()[0],
                "trainer": date_base.trainer.unique()[0]})
        body2 = ""
        for i in date_base.index:
            listpeserta = "-- " + date_base.peserta[i] + " - " + date_base.dept[i] + "\n"
            body2 += listpeserta
        body3 = "\n\nMohon bantuan rekan-rekan untuk dapat hadir tepat waktu, mengisi evaluasi training maupun posttest (bila ada).\n" \
        "Terimakasih ya\n" \
        "Salam\n\n" \
        "Generated by pyYDL any issue please inform prameswari.kristal@nutrifood.co.id"
        mail.Body = body1 + body2 + body3
        mail.Save()
        mail.Send()
        
        idx = date_base.index
        data.loc[idx, 'status_meetreq'] = 'done'

In [3]:
os.remove(join(os.getcwd(), 'datasets', 'meetreq_training.xlsx'))

writer = pd.ExcelWriter('datasets/meetreq_training.xlsx')
data.to_excel(writer, 'training')
writer.save()

ModuleNotFoundError: No module named 'openpyxl'